# Testing Race plot

In [1]:
import pandas as pd
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import os
import datetime

In [3]:
path = "\\".join(os.getcwd().split("\\")[:-2]) # + "\\common_data"
path

'C:\\Users\\Camila\\Documents\\git\\common_data'

In [4]:
# Importing the Excel file with all the cities
covid_munic = pd.read_csv(path+"\data\caso_full_with_indicators.csv")
covid_munic["date"] = pd.to_datetime(covid_munic['date'], format="%Y/%m/%d")
covid_munic["date_formatted"] = covid_munic["date"].apply(lambda x: x.strftime('%d/%m'))

region = {'AC': "Norte", 'AM': "Norte", 'AP': "Norte", 'PA': "Norte", 'RO': "Norte", 'RR': "Norte", 'TO': "Norte",
          'AL': "Nordeste", 'BA': "Nordeste", 'CE': "Nordeste", 'MA': "Nordeste", 'PB': "Nordeste", 'PE': "Nordeste", 
          'PI': "Nordeste", 'RN': "Nordeste", 'SE': "Nordeste", 
          'DF': "Centro Oeste", 'GO': "Centro Oeste", 'MS': "Centro Oeste", 'MT': "Centro Oeste",
          'ES': "Sudeste", 'MG': "Sudeste", 'RJ': "Sudeste", 'SP': "Sudeste",
          'PR': "Sul", 'SC': "Sul", 'RS': "Sul"}

covid_munic['Region'] = covid_munic['state'].map(region)

covid_munic['Death Rate'] = covid_munic['last_available_deaths'] / covid_munic['last_available_confirmed'] * 100

covid_munic.head(1)

,city,city_ibge_code,date,epidemiological_week,estimated_population_2019,is_last,is_repeated,last_available_confirmed,last_available_confirmed_per_100k_inhabitants,last_available_date,...,T_BANAGUA,T_DENS,AGUA_ESGOTO,T_RMAXIDOSO,IDHM,IDHM_E,IDHM_L,IDHM_R,date_formatted,Death Rate
0,Acrelândia,1200013.0,2020-06-09,24,15256.0,True,False,167,1094.65128,2020-06-09,...,55.29,41.61,23.82,2.54,0.604,0.466,0.808,0.584,09/06,1.197605


In [5]:
covid_uf = covid_munic[covid_munic["place_type"] == "state"].groupby(["state", "date"])["last_available_confirmed", "last_available_deaths", "new_confirmed", "new_deaths"].sum().reset_index()
covid_uf["date"] = pd.to_datetime(covid_uf['date'], format="%Y/%m/%d")
covid_uf["date_formatted"] = covid_uf["date"].apply(lambda x: x.strftime('%d/%m'))

# Create Region
region = {'AC': "Norte", 'AM': "Norte", 'AP': "Norte", 'PA': "Norte", 'RO': "Norte", 'RR': "Norte", 'TO': "Norte",
          'AL': "Nordeste", 'BA': "Nordeste", 'CE': "Nordeste", 'MA': "Nordeste", 'PB': "Nordeste", 'PE': "Nordeste", 
          'PI': "Nordeste", 'RN': "Nordeste", 'SE': "Nordeste", 
          'DF': "Centro Oeste", 'GO': "Centro Oeste", 'MS': "Centro Oeste", 'MT': "Centro Oeste",
          'ES': "Sudeste", 'MG': "Sudeste", 'RJ': "Sudeste", 'SP': "Sudeste",
          'PR': "Sul", 'SC': "Sul", 'RS': "Sul"}

covid_uf['Region'] = covid_uf['state'].map(region)

covid_uf['Death Rate'] = covid_uf['last_available_deaths'] / covid_uf['last_available_confirmed'] * 100

covid_uf.tail()

C:\Users\Camila\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,state,date,last_available_confirmed,last_available_deaths,new_confirmed,new_deaths,date_formatted,Region,Death Rate
2402,TO,2020-06-05,5505,96,807,14,05/06,Norte,1.743869
2403,TO,2020-06-06,5644,99,139,3,06/06,Norte,1.754075
2404,TO,2020-06-07,5644,99,0,0,07/06,Norte,1.754075
2405,TO,2020-06-08,6052,114,408,15,08/06,Norte,1.883675
2406,TO,2020-06-09,6052,114,0,0,09/06,Norte,1.883675


In [6]:
covid_uf[(covid_uf["new_confirmed"] > 0) & (covid_uf["new_deaths"] > 0)].groupby("date")["state"].count().tail(10)

date
2020-05-31    24
2020-06-01    25
2020-06-02    26
2020-06-03    25
2020-06-04    24
2020-06-05    24
2020-06-06    25
2020-06-07    25
2020-06-08    23
2020-06-09    22
Name: state, dtype: int64

In [7]:
confirmed = covid_uf.pivot_table(index=["state","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_confirmed':sum}, fill_value=0).reset_index()

In [8]:
confirmed.to_csv('confirmed_racebars.csv', encoding='utf-8', index=False)

In [9]:
death = covid_uf.pivot_table(index=["state","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_deaths':sum}, fill_value=0).reset_index()
death.to_csv('deaths_racebars.csv', encoding='utf-8', index=False)


In [10]:
death_rate = covid_uf.pivot_table(index=["state","Region"], columns=["date", "date_formatted"], aggfunc={'Death Rate':sum}, fill_value=0).reset_index()
death_rate.to_csv('death_rate_racebars.csv', encoding='utf-8', index=False)

In [11]:
confirmed_munic = covid_munic[covid_munic["place_type"] == "city"].pivot_table(index=["city_ibge_code","city","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_confirmed':max}, fill_value=0).reset_index()
confirmed_munic.to_csv('confirmed_munic_racebars.csv', encoding='utf-8', index=False)


In [12]:
death_munic = covid_munic[covid_munic["place_type"] == "city"].pivot_table(index=["city_ibge_code","city","Region"], columns=["date", "date_formatted"], aggfunc={'last_available_deaths':max}, fill_value=0).reset_index()
death_munic.to_csv('deaths_munic_racebars.csv', encoding='utf-8', index=False)
